In [1]:
import os

In [3]:
%pwd

'/Users/macbook/Desktop/MLOPs-Production-Ready-Machine-Learning-Project_Wine_Quality/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/Users/macbook/Desktop/MLOPs-Production-Ready-Machine-Learning-Project_Wine_Quality'

In [6]:
from dataclasses import dataclass       #entity (config_entity)
from pathlib import Path


@dataclass(frozen=True)          #dataclass is a decorator which doesnt require self.__init__ where you can directly initialize the varaiables also frozen = True means it wont be taking other variables expect the defined varaiables
class DataIngestionConfig:
    root_dir: Path            #get this paths from the config.yaml file
    source_URL: str            #get this paths from the config.yaml file
    local_data_file: Path        #get this paths from the config.yaml file
    unzip_dir: Path                #get this paths from the config.yaml file

In [13]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:       #it will give file paths and urls and returns in one single shot
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])   


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-05 12:50:10,768: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-12-05 12:50:10,773: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-05 12:50:10,778: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-05 12:50:10,781: INFO: common: created directory at: artifacts]
[2024-12-05 12:50:10,784: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-05 12:50:11,849: INFO: 3447005916: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 21984
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "85a38be3d4c70e3b6dc97fc47873623da88c9b850e1526aeb57ff236bdd2815c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 27E1:354370:1DCBA0:2557F7:6751542B
Accept-Ranges: bytes
Date: Thu, 05